# Atlas Vector Search - Local Retrieval-Augmented Generation (RAG)

This notebook is a companion for the [Local Retrieval-Augmented Generation (RAG)](https://www.mongodb.com/docs/atlas/atlas-vector-search/tutorials/local-rag/) page. Refer to the page for set up steps and explanation details.

In [ ]:
pip install --quiet pymongo gpt4all sentence_transformers

In [ ]:
ATLAS_CONNECTION_STRING = ("mongodb://localhost:<port-number>/?directConnection=true")

In [ ]:
from pymongo import MongoClient
from sentence_transformers import SentenceTransformer

# Connect to your local Atlas deployment or Atlas Cluster
client = MongoClient(ATLAS_CONNECTION_STRING)

# Select the sample_airbnb.listingsAndReviews collection
collection = client["sample_airbnb"]["listingsAndReviews"]

# Load the embedding model (https://huggingface.co/sentence-transformers/mixedbread-ai/mxbai-embed-large-v1)
model_path = "<model-path>"
model = SentenceTransformer('mixedbread-ai/mxbai-embed-large-v1')
model.save(model_path)
model = SentenceTransformer(model_path)

# Define function to generate embeddings
def get_embedding(text):
    return model.encode(text).tolist()

# Filters for only documents with a summary field and without an embeddings field
filter = { '$and': [ { 'summary': { '$exists': True, '$ne': None } }, { 'embeddings': { '$exists': False } } ] }

# Creates embeddings for subset of the collection
updated_doc_count = 0
for document in collection.find(filter).limit(50):
    text = document['summary']
    embedding = get_embedding(text)
    collection.update_one({ '_id': document['_id'] }, { "$set": { 'embeddings': embedding } }, upsert=True)
    updated_doc_count += 1

print("Documents updated: {}".format(updated_doc_count))


In [ ]:
from pymongo.operations import SearchIndexModel

# Create your index model, then create the search index
search_index_model = SearchIndexModel(
  definition = {
    "fields": [
      {
        "type": "vector",
        "numDimensions": 1024,
        "path": "embeddings",
        "similarity": "cosine"
      }
    ]
  },
  name = "vector_index",
  type = "vectorSearch" 
)
collection.create_search_index(model=search_index_model)


In [ ]:
# Function to get the results of a vector search query
def get_query_results(query):
   query_embedding = get_embedding(query)

   pipeline = [
      {
            "$vectorSearch": {
               "index": "vector_index",
               "queryVector": query_embedding,
               "path": "embeddings",
               "exact": True,
               "limit": 5
            }
      }, {
            "$project": {
               "_id": 0,
               "summary": 1,
               "listing_url": 1,
               "score": {
                  "$meta": "vectorSearchScore"
               }
            }
      }
   ]

   results = collection.aggregate(pipeline)

   array_of_results = []
   for doc in results:
      array_of_results.append(doc)
   return array_of_results

In [ ]:
import pprint
pprint.pprint(get_query_results("beach house"))

In [ ]:
from gpt4all import GPT4All

# Download the model and move it to the same directory as this notebook
# For complete details, refer to the documentation page
local_llm_path = "./mistral-7b-openorca.gguf2.Q4_0.gguf"
local_llm = GPT4All(local_llm_path)

In [ ]:
question = "Can you recommend a few AirBnBs that are beach houses? Include a link to the listing."
documents = get_query_results(question)

text_documents = ""
for doc in documents:
    summary = doc.get("summary", "")
    link = doc.get("listing_url", "")
    string = f"Summary: {summary} Link: {link}. \n"
    text_documents += string

prompt = f"""Use the following pieces of context to answer the question at the end.
    {text_documents}
    Question: {question}
"""

response = local_llm.generate(prompt)
cleaned_response = response.replace('\\n', '\n')
print(cleaned_response)